In [1]:
pip install timm

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.4 MB 4.2 MB/s eta 0:00:01
   ---------------------- ----------------- 1.3/2.4 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
from timm import create_model
from torchvision import transforms
from PIL import Image

# Load ViT pretrained on ImageNet
model = create_model('vit_base_patch16_224', pretrained=True)
model.eval()

# Load and preprocess image
image = Image.open("C:/Users/Admin/OneDrive/Desktop/vscode/fyp/skinType/patches128x128")
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])
img_tensor = transform(image).unsqueeze(0)

# Inference
with torch.no_grad():
    logits = model(img_tensor)

print("Logits shape:", logits.shape)  # [1, 1000] for 1000 ImageNet classes


c:\Users\Admin\OneDrive\Desktop\vscode\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 